In [ ]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

In [ ]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
# Included the 'SPECIAL_CONSIDERATIONSS' in the dropped collumns in an attempt to optimize the model.
#Special considerations is not a specific enough feature to dramatically effect the model so we will drop it in an attempt to increase the weighting of more practical business features.
application_df = application_df.drop(columns= ['EIN', 'NAME'])

In [ ]:
# Determine the number of unique values in each column.
application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [ ]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [ ]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
# For this excercise my value count cut-off was 25
application_to_replace = []
app_value_count = application_df['APPLICATION_TYPE'].value_counts().reset_index()
replace_app = app_value_count[app_value_count['APPLICATION_TYPE'] < 25]

application_to_replace_to_replace = replace_app['index'].tolist()
# Replace in dataframe
for app in application_to_replace:
  application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")


In [ ]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#  Cut-Off: 110
classifications_to_replace = []
class_value_counts_2 = application_df['CLASSIFICATION'].value_counts().reset_index()
replace_class = class_value_counts_2[class_value_counts_2['CLASSIFICATION'] <= 60]

classifications_to_replace = replace_class['index'].tolist()

# Replace in dataframe
for cls in classifications_to_replace:
  application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      499
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
Name: CLASSIFICATION, dtype: int64

In [ ]:
application_df_dummies = pd.get_dummies(application_df, columns= ['INCOME_AMT', 'CLASSIFICATION', 'APPLICATION_TYPE',  'ORGANIZATION', 'AFFILIATION', "USE_CASE", 'SPECIAL_CONSIDERATIONS'])

In [ ]:
# Split our preprocessed data into our features and target arrays
features_df = application_df_dummies.drop(columns=['IS_SUCCESSFUL'])
target_df = application_df_dummies['IS_SUCCESSFUL']
X = features_df.values
y = target_df.values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=59, activation="relu", input_dim=59))


# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=59, activation="relu"))

# Third hidden layer
#Added an additional hidden layer to increase the number of nodes in the model and attempt to increase model fit.
#This addition does not overly crowd the hyperparameters as it sticks to best practice of 2-4 hidden layers but attempts to increase the models learning.
nn_model.add(tf.keras.layers.Dense(units=59, activation="elu"))
nn_model.add(tf.keras.layers.Dense(units=59, activation="elu"))
# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_59 (Dense)            (None, 59)                3540      
                                                                 
 dense_60 (Dense)            (None, 59)                3540      
                                                                 
 dense_61 (Dense)            (None, 59)                3540      
                                                                 
 dense_62 (Dense)            (None, 59)                3540      
                                                                 
 dense_63 (Dense)            (None, 1)                 60        
                                                                 
Total params: 14,220
Trainable params: 14,220
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
# Train the model
#Reduced number of Epochs to 50 from 100 as previous model seemed to reach marginal utility of training around 50 epochs
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 4s 3ms/step - loss: 0.5702 - accuracy: 0.7214
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5553 - accuracy: 0.7292
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5510 - accuracy: 0.7315
Epoch 4/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5496 - accuracy: 0.7306
Epoch 5/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5467 - accuracy: 0.7313
Epoch 6/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5462 - accuracy: 0.7328
Epoch 7/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5453 - accuracy: 0.7333
Epoch 8/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5444 - accuracy: 0.7327
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5435 - accuracy: 0.7332
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5431 - accuracy: 0.7348

In [ ]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5591 - accuracy: 0.7304 - 509ms/epoch - 2ms/step
Loss: 0.5591038465499878, Accuracy: 0.7303789854049683


In [ ]:
from keras.saving.saving_api import load_model
# Export our model to HDF5 file
nn_model.save('AlphabetSoupCharity_Optimization.h5', save_format= 'h5')
nn_model = load_model('AlphabetSoupCharity_Optimization.h5')